In [1]:
import numpy as np
import cv2
import math
import collections
import os
from pathlib import Path
from skimage.morphology import skeletonize

In [2]:
import matplotlib.pyplot as plt

%matplotlib inline
def plot_image(img, zoom=1.5):
    assert(2 <= len(img.shape) <= 3)
    
    is_gray = len(img.shape) == 2
    if not is_gray:
        img = img[:,:,::-1]
    n_len = 6.5
    
    n, m = img.shape[:2]
    frac = n / float(m)
    n, m = zoom * frac * n_len, zoom * n_len
    
    fig = plt.figure(figsize=(n, m))
    ax = fig.add_subplot(111)
    ax.imshow(img, cmap='gray' if is_gray else None)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    plt.axis("off")
    plt.show()

In [3]:
def skeleton_endpoints(skeleton, neighbours=1):
    skeleton = skeleton.copy()
    skeleton[skeleton > 0] = 1
    skeleton = skeleton.astype(np.uint8)

    kernel = np.array(
        [[1,  1, 1],
         [1, 10, 1],
         [1,  1, 1]], dtype=np.uint8)
    filtered = cv2.filter2D(skeleton, -1, kernel)

    result = np.zeros(skeleton.shape)
    result[np.where(filtered == 10 + neighbours)] = 1
    return result

In [4]:
def create_brush(d):
    brush = np.zeros((d, d), dtype=np.uint8)
    cv2.circle(brush, (d // 2, d // 2), d // 2, 1, -1)
    return brush

In [5]:
def scale_image_pixels(img):
    img = img.astype(np.float32)
    img = (img - img.min()) / (img.max() - img.min()) * 255
    return img.astype(np.uint8)

In [6]:
def read(pattern="[2-5].jpg"):
    images = []
    
    folder = Path("samples")
    files_with_maps = folder.glob(pattern)
    files=list(files_with_maps)
    
    for file in files:
        img = cv2.imread(str(file), 0)
        images.append(img)
        
    return images

In [7]:
def skeleton_ex_points(img,rad=55):
    end_points=[]
    _, img_mask = cv2.threshold(img, 80, 255, cv2.THRESH_BINARY_INV)
    img = cv2.dilate(img_mask, kernel=create_brush(13), iterations=2)
    skeleton = skeletonize(img > 0)
    points = skeleton_endpoints(skeleton)
    points =np.where(points!=0)
    
    skel_pix=skeleton.copy()
    skel_pix = scale_image_pixels(skel_pix)
    skel_pix = cv2.dilate(skel_pix, create_brush(9))
    skel_pix = scale_image_pixels(skel_pix)
    skel_pix=cv2.bitwise_not(skel_pix)
    
    skeleton=skeleton.astype(np.uint8)*255
    skeleton=cv2.bitwise_not(skeleton)    

    for i in range(len(points[0])):
        white=np.ones(skel_pix.shape).astype(np.uint8)*255
        cv2.circle(white, (points[1][i], points[0][i]), rad, (0, 0 , 0), 2)    
        cnt_l=cv2.findContours(white, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0][0]
        cnt_r=cv2.findContours(white, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0][1]
        area_r=cv2.contourArea(cnt_r)
        area_l=cv2.contourArea(cnt_l)
        if area_r>area_l:
            if area_l/(math.pi*rad*rad)>0.6:
                end_points.append(np.array([points[1][i], points[0][i]]))
        else:
            if area_r/(math.pi*rad*rad)>0.6:
                end_points.append(np.array([points[1][i], points[0][i]])) 
    return skeleton, skel_pix, end_points

In [8]:
def center_balls (img):
    #img = cv2.medianBlur(img_gray,101)
    img=cv2.bitwise_not(img)
    #img=img_gray.copy()
    cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
    #plot_image(cimg)
    circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,10,param1=100,param2=22,minRadius=7,maxRadius=20)
    circles = np.int64(np.around(circles))
    balls = list(circles[0][:,0:2])
    return balls
    

In [9]:
def remove (points,eps=27):
    new_points=points.copy()
    for i in range(len(points)):
        p=points[i]
        check=False
        for j in range(i+1,len(points)):
            c=points[j]  
            if(np.linalg.norm(p-c)<eps):
                check=True
        if check:
            new_points=list(filter(lambda a: a[0]!=p[0] and a[1]!=p[1], new_points))
    return new_points
    

In [10]:
def vector (points, balls, skel,rad=55,eps=27):
    new_points=remove(points)
    new_balls=remove(balls) 
    vertexes=new_balls.copy()
    vec=[0,0,0,0,0,0,0,0,0]
    for p in new_points:
        check=True
        for c in new_balls:
            if(np.linalg.norm(p-c)<eps):
                check=False
        if check:
            vertexes.append(p)
    for v in vertexes:
        white=np.ones(skel.shape).astype(np.uint8)*255
        cv2.circle(white,(v[0],v[1]),rad,(0,0,0),2)   
        ind=np.where(white==0)
        j=round(len(np.where(skel[ind]==0)[0])/24)-1
        vec[j]+=1
        #plot_image(white==skel,zoom=0.5)
        #print(j+1, "        ",v[0],v[1])
    return vec


In [11]:
#Обучение
images=read(pattern="[2-5].jpg")
vectors=[]
for img in images:
    skeleton, skel, points = skeleton_ex_points(img,rad=55)
    balls=center_balls(img)
    vec =vector(points, balls, skel)
    vectors.append(vec)
vectors=np.array(vectors)

In [12]:
ans = ['1st','2nd','3rd','4th']
col=list("123456789")
import pandas as pd
data = pd.DataFrame(ans)
for i in col:
    data[i] = vectors[:,int(i)-1]

y = data[0]
X = data[col]


In [13]:
data

,0,1,2,3,4,5,6,7,8,9
0,1st,3,4,3,3,0,0,0,0,0
1,2nd,4,5,4,1,0,0,0,0,0
2,3rd,4,3,5,2,1,0,0,0,0
3,4th,6,3,4,2,0,0,0,0,0


In [14]:
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression()
clf1.fit(X,y)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [15]:
num=[13,25,19,7]
vectors=[]
for d in num:
    images=read(pattern=str(d)+".jpg")
    for img in images:
        skeleton, skel, points = skeleton_ex_points(img,rad=55)
        balls=center_balls(img)
        vec=vector(points, balls, skel)
        vectors.append(vec)
vectors=np.array(vectors)

In [16]:
clf1.predict(vectors)

array(['1st', '2nd', '3rd', '4th'], dtype=object)

In [17]:
vectors

array([[3, 4, 3, 3, 0, 0, 0, 0, 0],
       [4, 5, 4, 1, 0, 0, 0, 0, 0],
       [4, 3, 5, 2, 1, 0, 0, 0, 0],
       [6, 3, 4, 2, 0, 0, 0, 0, 0]])